### Load packages etc, and load the child-class of gmspython, "Abatement", and load "techdata_to_tree" which converts a technology catalog into production trees

In [1]:
clean_up=True # removes gams-related files in work-folder if true
%run StdPackages.ipynb
os.chdir(py['main'])
import abatement, sys, ShockFunction
os.chdir(curr)
data_folder = os.getcwd()+'\\Data'
gams_folder = data_folder + "\\..\\gamsmodels\\Main"
# pickle_path = 

In [2]:
#NEW LOAD HERE before it works. Write m.export() in _calibration script.
#m = abatement.abate(pickle_path=gams_folder+"/gmspython_Abatement", work_folder=work_folder)

## A shock to the oil price
We present graphically what happens when the oil price gradually increases (static model solution for each price).
For this, we again use the sneaky solve, but we make sure to extract variables of interest for each iteration, in contrast to above where we were not interested in the solutions per se, but simply that the values of $\sigma$ ended in the desired spots.

In [121]:
#mideleritid start fra starten
#m.model.settings.databases["Abatement_0"]["sigma"].vals.loc[:] = 0.1
#m.model.settings.databases["Abatement_0"]["eta"].vals.loc[:] = -0.1

In [122]:
# m.write_and_run(name="calibrated", add_checkpoint="calibrated")

In [123]:
m.model_instances["calibrated"].out_db.get("sigma")

n
C_CO2_1             0.50
C_EH_1              1.20
C_EH_2              1.20
C_EH_3              1.20
C_EH_base           0.01
C_EL_1              1.20
C_EL_2              1.20
C_EL_3              1.20
C_EL_4              1.20
C_EL_5              1.20
C_EL_base           0.01
C_ER_1              1.20
C_ER_2              1.20
C_ER_base           0.01
EH                  0.01
EL                  0.01
EOP_tau1            0.50
ER                  0.01
ID_1                0.01
ID_2                0.01
ID_3                0.01
ID_4                0.01
ID_5                0.01
U_ID_C_EH_1_base    0.01
U_ID_C_EH_2_base    0.01
U_ID_C_EH_3_base    0.01
U_ID_C_EL_1_base    0.01
U_ID_C_EL_2_base    0.01
U_ID_C_EL_3_base    0.01
U_ID_C_EL_4_base    0.01
U_ID_C_EL_5_base    0.01
U_ID_C_ER_1_base    0.01
U_ID_C_ER_2_base    0.01
Name: sigma, dtype: float64

In [124]:
out_db = m.model_instances["calibrated"].out_db

In [125]:
m.write_and_run(name="oilshock")
#db_out = m.model_instances["baseline"].out_db

In [126]:
#m.write_and_run(name='taxshock', add_checkpoint='taxshock', overwrite=True)
mi = m.model_instances['oilshock']

In [127]:
#mi.out_db.series.__dict__

*2: Define structure of shock* 

In [128]:
db_end_of_loop = DataBase.GPM_database()

In [129]:
cond = out_db.get("PwT").index.str.contains("oil")
db_end_of_loop["PwT"] = out_db.get("PwT")[cond] + 12
#end_prices[cond] = out_db.get("PwT")[cond] + 5
#end_prices = end_prices[cond]

In [130]:
db_end_of_loop.series["PwT"].vals

n
C_EH_base_oil           13.0
C_EL_base_oil           13.0
C_ER_base_oil           13.0
ID_1_oil                13.0
ID_2_oil                13.0
ID_5_oil                13.0
U_ID_C_EH_1_base_oil    13.0
U_ID_C_EH_2_base_oil    13.0
U_ID_C_EH_3_base_oil    13.0
U_ID_C_EL_1_base_oil    13.0
U_ID_C_EL_2_base_oil    13.0
U_ID_C_EL_3_base_oil    13.0
U_ID_C_EL_4_base_oil    13.0
U_ID_C_EL_5_base_oil    13.0
U_ID_C_ER_1_base_oil    13.0
U_ID_C_ER_2_base_oil    13.0
Name: PwT, dtype: float64

In [131]:
#NY ME DDIREKTE
#m.model_instances["oilshock"].solve_sneakily(db_star=db_end_of_loop, from_cp=True, cp_init=m.checkpoints["calibrated"])

In [132]:
(shock_db, shock_kwargs) = ShockFunction.sneaky_db(out_db, db_end_of_loop, n_steps = 25, loop_name='oil_loop')

*3: Define the solution we wish to store for each iteration (conditions argument is optional, but here we extract only the production sector goods):*

In [133]:
shock_db.series["PwT_oil_loop_subset"].vals

Index(['C_EH_base_oil', 'C_EL_base_oil', 'C_ER_base_oil', 'ID_1_oil',
       'ID_2_oil', 'ID_5_oil', 'U_ID_C_EH_1_base_oil', 'U_ID_C_EH_2_base_oil',
       'U_ID_C_EH_3_base_oil', 'U_ID_C_EL_1_base_oil', 'U_ID_C_EL_2_base_oil',
       'U_ID_C_EL_3_base_oil', 'U_ID_C_EL_4_base_oil', 'U_ID_C_EL_5_base_oil',
       'U_ID_C_ER_1_base_oil', 'U_ID_C_ER_2_base_oil'],
      dtype='object', name='n')

In [134]:
store_sol = {'qD': {'domains': shock_kwargs['loop_name']}}

*4: Create Shock instance*

In [135]:
shock = mi.std_UEVAS_from_db(shock_db, loop_name=shock_kwargs['loop_name'], shock_name=shock_kwargs['shock_name'], store_sol=store_sol)
#shock = mi.std_UEVAS_from_db(shock_db, loop_name=shock_kwargs['loop_name'], shock_name=shock_kwargs['shock_name'])
#solve_sneakily skal have selve checkpointet
#Gør noget så sneaky solve laver et sidste checkpoint

*5: Execute shock from the 'calibrated' checkpoint i.e. the model state after the sigmas have been chosen:*

In [136]:
mi.execute_shock_from_cp(shock=shock, cp=m.checkpoints['calibrated'])

In [137]:
sol_qD = mi.out_db.series["sol_qD"].vals

In [138]:
#tech_names = ["ID_thighest", "ID_thigher", "ID_tmedium", "ID_tlower", "ID_tlowest"]
#sol_qD.loc(axis=0)["oil_loop_1", tech_names]

In [139]:
sol_qD.reset_index().to_excel(os.getcwd() + "\\oil_price_shock_" + inputfile.split("_")[-1], index=False)

In [140]:
#m.model_instances["oilshock"].solve_sneakily(db_star=db_end_of_loop, from_cp=True, cp_init=m.checkpoints["oilshock"])

The solution is stored as a parameter noted as "sol_"+ the name of the variable. In our case, the specific quantities are stored:

### **3.1 Diagnostics (dated)**
When the model cannot be solved, we run some diagnostics:

In [ ]:
# for symbol in ["PbT", "PwT", "qD"]:
#     start_vals = m.model_instances["baseline"].out_db.series["load_" + symbol].vals
#     end_vals = m.model_instances["baseline"].out_db.series[symbol].vals
#     rel_changes = (end_vals - start_vals)/start_vals
#     large_changes = rel_changes[~(rel_changes.between(-0.0001, 0.0001))]
#     if len(large_changes) > 0:
#         print("Large deviations for symbol \n" + symbol + ":")
#         print(large_changes, "\n")
#     else:
#         print("No large deviations for symbol: " + symbol + ".\n")

In [ ]:
# from decimal import Decimal

In [ ]:
# #Demands
# print(Decimal(qD["U_EOP_tau1_1"] - mu[("U_EOP_tau1_1", "C_CO2_1")] * qS["C_CO2_1"]))
# print(Decimal(qD["EOP_tau1"] - mu.loc[("U_EOP_tau1_1", "EOP_tau1")] * qD["U_EOP_tau1_1"]))
# print(Decimal(qD["EOP_tau1_electricity"] - mu[("EOP_tau1_electricity", "EOP_tau1")] * qD["EOP_tau1"]))
# print(Decimal(qD["EOP_tau1_K"] - mu[("EOP_tau1_K", "EOP_tau1")] * qD["EOP_tau1"]))
# #zero profits
# print(Decimal(qS["C_CO2_1"]*PbT["C_CO2_1"] - qD["U_EOP_tau1_1"]*PwT["U_EOP_tau1_1"]))
# print(Decimal(qD["U_EOP_tau1_1"]*PwT["U_EOP_tau1_1"] - qD["EOP_tau1"]*PwT["EOP_tau1"]))
# print(Decimal(qD["EOP_tau1"]*PwT["EOP_tau1"] - qD["EOP_tau1_electricity"]*PwT["EOP_tau1_electricity"] - qD["EOP_tau1_K"]*PwT["EOP_tau1_K"]))